# Customer product recommendation system

## Load Dependencies and dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('Online_Retail.xlsx')
print(df.head())

  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  


In [3]:
df = df.loc[df['Quantity'] > 0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513135 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      513135 non-null  object        
 1   StockCode    513135 non-null  object        
 2   Description  512034 non-null  object        
 3   Quantity     513135 non-null  int64         
 4   InvoiceDate  513135 non-null  datetime64[ns]
 5   Price        513135 non-null  float64       
 6   Customer ID  407695 non-null  float64       
 7   Country      513135 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 35.2+ MB


In [4]:
df['Customer ID'].isna().sum()
df = df.dropna(subset=['Customer ID'])

## Create matrix/map

In [5]:
customer_item_matrix = df.pivot_table(
    index='Customer ID',
    columns='StockCode',
    values='Quantity',
    aggfunc='sum'
)
customer_item_matrix.loc[12481:].head()

StockCode,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,...,ADJUST2,BANK CHARGES,C2,D,M,PADS,POST,SP1002,TEST001,TEST002
Customer ID,,,,,,,,,,,,,,,,,,,,,
12481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN
12482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
12483.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN
12484.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN
12485.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(customer_item_matrix.shape)
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)

(4314, 4017)


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
user_user_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix))
user_user_sim_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,4304,4305,4306,4307,4308,4309,4310,4311,4312,4313
0,1.000000,0.000000,0.000000,0.144707,0.000000,0.000000,0.000000,0.000000,0.000000,0.183211,...,0.226455,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.071878,0.000000
1,0.000000,1.000000,0.053452,0.025198,0.052164,0.028172,0.026726,0.025482,0.101460,0.083744,...,0.069007,0.000000,0.0,0.053452,0.075593,0.047544,0.090351,0.0,0.087612,0.027242
2,0.000000,0.053452,1.000000,0.023570,0.000000,0.000000,0.000000,0.000000,0.189814,0.017408,...,0.032275,0.000000,0.0,0.000000,0.000000,0.053368,0.000000,0.0,0.000000,0.000000
3,0.144707,0.025198,0.023570,1.000000,0.046004,0.049690,0.094281,0.044947,0.140610,0.196946,...,0.076073,0.054433,0.0,0.000000,0.033333,0.109017,0.019920,0.0,0.064389,0.060062
4,0.000000,0.052164,0.000000,0.046004,1.000000,0.051434,0.048795,0.046524,0.026463,0.033976,...,0.000000,0.000000,0.0,0.000000,0.000000,0.034721,0.041239,0.0,0.026660,0.024868


In [9]:
user_user_sim_matrix.columns = customer_item_matrix.index

user_user_sim_matrix['CustomerID'] = customer_item_matrix.index
user_user_sim_matrix = user_user_sim_matrix.set_index('CustomerID')
user_user_sim_matrix.head()

Customer ID,12346.0,12347.0,12348.0,12349.0,12351.0,12352.0,12353.0,12355.0,12356.0,12357.0,...,18277.0,18278.0,18279.0,18280.0,18281.0,18283.0,18284.0,18285.0,18286.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,1.000000,0.000000,0.000000,0.144707,0.000000,0.000000,0.000000,0.000000,0.000000,0.183211,...,0.226455,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.071878,0.000000
12347.0,0.000000,1.000000,0.053452,0.025198,0.052164,0.028172,0.026726,0.025482,0.101460,0.083744,...,0.069007,0.000000,0.0,0.053452,0.075593,0.047544,0.090351,0.0,0.087612,0.027242
12348.0,0.000000,0.053452,1.000000,0.023570,0.000000,0.000000,0.000000,0.000000,0.189814,0.017408,...,0.032275,0.000000,0.0,0.000000,0.000000,0.053368,0.000000,0.0,0.000000,0.000000
12349.0,0.144707,0.025198,0.023570,1.000000,0.046004,0.049690,0.094281,0.044947,0.140610,0.196946,...,0.076073,0.054433,0.0,0.000000,0.033333,0.109017,0.019920,0.0,0.064389,0.060062
12351.0,0.000000,0.052164,0.000000,0.046004,1.000000,0.051434,0.048795,0.046524,0.026463,0.033976,...,0.000000,0.000000,0.0,0.000000,0.000000,0.034721,0.041239,0.0,0.026660,0.024868


In [10]:
user_user_sim_matrix.loc[12348.0].sort_values(ascending=False).head(10)

Customer ID
12348.0    1.000000
15008.0    0.456435
17940.0    0.372678
13027.0    0.338062
12661.0    0.337100
17407.0    0.326377
14153.0    0.325396
12504.0    0.316228
12634.0    0.300000
14316.0    0.292770
Name: 12348.0, dtype: float64

## Make recommendations 

In [11]:
user_user_sim_matrix.loc[12348.0].sort_values(ascending=False)
items_bought_by_A = customer_item_matrix.loc[12348.0][customer_item_matrix.loc[12348.0]>0]
print("Items Bought by A: ")
print(items_bought_by_A)

Items Bought by A: 
StockCode
21211    1
21213    1
21723    1
21725    1
21726    1
21967    1
21977    1
21980    1
21981    1
21982    1
21983    1
21985    1
22950    1
22951    1
22952    1
84987    1
84988    1
84991    1
84992    1
POST     1
Name: 12348.0, dtype: int64


In [12]:
items_bought_by_B = customer_item_matrix.loc[17935.0][customer_item_matrix.loc[17935.0]>0]
print("Items bought by B:")
print(items_bought_by_B)

print()

items_to_recommend_to_B = set(items_bought_by_A.index) - set(items_bought_by_B.index)
print("Items to Recommend to B ")
print(items_to_recommend_to_B)
df.loc[df['StockCode'].isin(items_to_recommend_to_B),['StockCode', 'Description']].drop_duplicates().set_index('StockCode')

Items bought by B:
StockCode
20713     1
20828     1
20983     1
21930     1
21931     1
21934     1
22200     1
22202     1
22304     1
22305     1
22423     1
22473     1
22474     1
22475     1
22513     1
22633     1
22654     1
22697     1
22698     1
22699     1
22759     1
22834     1
22837     1
22890     1
22906     1
85214     1
47599B    1
48173C    1
75149N    1
82001S    1
85049E    1
85099C    1
85232B    1
Name: 17935.0, dtype: int64

Items to Recommend to B 
{84992, 22950, 22951, 22952, 21967, 21977, 21723, 21980, 21213, 21982, 21726, 21725, 21985, 21981, 21211, 21983, 'POST', 84987, 84988, 84991}


,Description
StockCode,
POST,POSTAGE
84988,SET OF 72 PINK HEART PAPER DOILIES
21977,PACK OF 60 PINK PAISLEY CAKE CASES
84991,60 TEATIME FAIRY CAKE CASES
21726,MULTI HEARTS STICKERS
21985,PACK OF 12 HEARTS DESIGN TISSUES
21980,PACK OF 12 RED SPOTTY TISSUES
21967,PACK OF 12 SKULL TISSUES
84992,72 SWEETHEART FAIRY CAKE CASES


In [13]:
item_item_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix.T))
item_item_sim_matrix.columns = customer_item_matrix.T.index

item_item_sim_matrix['StockCode'] = customer_item_matrix.T.index
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')

print(item_item_sim_matrix)

StockCode     10002     10080  10109     10120     10125     10133     10134  \
StockCode                                                                      
10002      1.000000  0.040423    0.0  0.070367  0.057166  0.029412  0.036564   
10080      0.040423  1.000000    0.0  0.087039  0.000000  0.000000  0.000000   
10109      0.000000  0.000000    1.0  0.000000  0.000000  0.000000  0.000000   
10120      0.070367  0.087039    0.0  1.000000  0.024618  0.042220  0.000000   
10125      0.057166  0.000000    0.0  0.024618  1.000000  0.068599  0.042640   
...             ...       ...    ...       ...       ...       ...       ...   
PADS       0.000000  0.000000    0.0  0.000000  0.081650  0.000000  0.000000   
POST       0.095954  0.000000    0.0  0.022957  0.027975  0.015992  0.000000   
SP1002     0.114332  0.000000    0.0  0.123091  0.100000  0.085749  0.000000   
TEST001    0.000000  0.000000    0.0  0.000000  0.000000  0.000000  0.000000   
TEST002    0.000000  0.000000    0.0  0.

In [14]:
top_10_similar_items = list(item_item_sim_matrix.loc[21213].sort_values(ascending=False).iloc[:10].index)

print(top_10_similar_items)
print()
print(df.loc[
    df['StockCode'].isin(top_10_similar_items),
    ['StockCode', 'Description']
].drop_duplicates().set_index('StockCode').loc[top_10_similar_items])

[21213, 21975, 22417, 21212, 84991, 84992, 21977, 21989, 21976, 21211]

                                  Description
StockCode                                    
21213             PACK OF 72 SKULL CAKE CASES
21975          PACK OF 60 DINOSAUR CAKE CASES
22417          PACK OF 60 SPACEBOY CAKE CASES
21212        PACK OF 72 RETRO SPOT CAKE CASES
21212         PACK OF 72 RETROSPOT CAKE CASES
84991             60 TEATIME FAIRY CAKE CASES
84992          72 SWEETHEART FAIRY CAKE CASES
21977      PACK OF 60 PINK PAISLEY CAKE CASES
21989          PACK OF 20 SKULL PAPER NAPKINS
21976          PACK OF 60 MUSHROOM CAKE CASES
21211          SET OF 72 SKULL PAPER  DOILIES


C:\python\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
